### Fetching the Data from various platforms (PubChem, OpenFDA, RXnorms) and Drugbank Dataset

In [18]:
from concurrent.futures import ThreadPoolExecutor
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import json
import ssl
import xml.etree.ElementTree as ET
import logging

# Disable SSL certificate validation for SPARQLWrapper
ssl._create_default_https_context = ssl._create_unverified_context

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

class DrugDataFetcher:
    def __init__(self, drugbank_xml_path):
        self.pubchem_rest_endpoint = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
        self.openfda_rest_endpoint = "https://api.fda.gov/drug/"
        self.rxnorm_rest_endpoint = "https://rxnav.nlm.nih.gov/REST"
        self.drugbank_xml_path = drugbank_xml_path
        try:
            self.drugbank_tree = ET.parse(self.drugbank_xml_path)
            self.drugbank_root = self.drugbank_tree.getroot()
            self.namespace = {"db": "http://www.drugbank.ca"}
            logging.info("DrugBank XML successfully loaded.")
        except Exception as e:
            logging.error(f"Error loading DrugBank XML: {e}")
            raise

    def fetch_pubchem_data(self, compound_name):
        url = f"{self.pubchem_rest_endpoint}/compound/name/{compound_name}/JSON"
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.json()
            else:
                logging.warning(f"PubChem API error: {response.status_code}")
                return None
        except Exception as e:
            logging.error(f"Error fetching PubChem data: {e}")
            return None

    def fetch_openfda_data(self, query):
        url = f"{self.openfda_rest_endpoint}label.json"
        params = {"search": query, "limit": 1}
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                logging.warning(f"OpenFDA API error: {response.status_code}")
                return None
        except Exception as e:
            logging.error(f"Error fetching OpenFDA data: {e}")
            return None

    def fetch_rxnorm_data(self, drug_name):
        url = f"{self.rxnorm_rest_endpoint}/rxcui.json"
        params = {"name": drug_name}
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                logging.warning(f"RxNorm API error: {response.status_code}")
                return None
        except Exception as e:
            logging.error(f"Error fetching RxNorm data: {e}")
            return None

    def fetch_drugbank_data(self, user_input):
        general_drug_info = None
        matching_products = []
        for drug in self.drugbank_root.findall("db:drug", self.namespace):
            name = drug.find("db:name", self.namespace)
            if name is not None:
                if user_input.lower() == name.text.lower():
                    general_drug_info = {
                        "id": drug.find("db:drugbank-id[@primary='true']", self.namespace).text,
                        "name": name.text,
                        "description": self.get_text(drug, "db:description"),
                        "indication": self.get_text(drug, "db:indication"),
                        "mechanism_of_action": self.get_text(drug, "db:mechanism-of-action"),
                        "toxicity": self.get_text(drug, "db:toxicity"),
                    }
                    break
                for product in drug.findall("db:products/db:product", self.namespace):
                    product_name = self.get_text(product, "db:name")
                    if product_name and user_input.lower() == product_name.lower():
                        if not general_drug_info:
                            general_drug_info = {
                                "id": drug.find("db:drugbank-id[@primary='true']", self.namespace).text,
                                "name": name.text,
                                "description": self.get_text(drug, "db:description"),
                                "indication": self.get_text(drug, "db:indication"),
                                "mechanism_of_action": self.get_text(drug, "db:mechanism-of-action"),
                                "toxicity": self.get_text(drug, "db:toxicity"),
                            }
                        matching_products.append({
                            "product_name": product_name,
                            "general_drug_id": drug.find("db:drugbank-id[@primary='true']", self.namespace).text,
                            "general_drug_name": name.text,
                            "labeller": self.get_text(product, "db:labeller"),
                            "strength": self.get_text(product, "db:strength"),
                            "route": self.get_text(product, "db:route"),
                            "approved": self.get_text(product, "db:approved"),
                            "country": self.get_text(product, "db:country"),
                        })
        result = {}
        if general_drug_info:
            result["general_drug_info"] = general_drug_info
        if matching_products:
            result["matching_products"] = matching_products
        return result

    def get_text(self, parent, tag):
        element = parent.find(tag, self.namespace)
        return element.text if element is not None else None


def fetch_all_data(fetcher, drug_name):
    """Fetch all data sources for a given drug."""
    drugbank_data = fetcher.fetch_drugbank_data(drug_name)
    pubchem_data = fetcher.fetch_pubchem_data(drug_name)
    openfda_data = fetcher.fetch_openfda_data(drug_name)
    rxnorm_data = fetcher.fetch_rxnorm_data(drug_name)
    return {
        "drugbank": drugbank_data,
        "pubchem": pubchem_data,
        "openfda": openfda_data,
        "rxnorm": rxnorm_data,
    }

# Initialize the DrugDataFetcher with the path to the DrugBank XML file
drugbank_xml_path = "drugbank.xml"  # Replace with your actual path
fetcher = DrugDataFetcher(drugbank_xml_path)

# if __name__ == "__main__":
#     # Initialize the DrugDataFetcher with the path to the DrugBank XML file
#     drugbank_xml_path = "drugbank.xml"  # Replace with your actual path
#     fetcher = DrugDataFetcher(drugbank_xml_path)

#     # Simulate user input for two drugs
#     drug_a = input("Enter the first drug or product name: ").strip()
#     drug_b = input("Enter the second drug or product name: ").strip()

#     # Use ThreadPoolExecutor to fetch data for both drugs in parallel
#     with ThreadPoolExecutor() as executor:
#         future_a = executor.submit(fetch_all_data, fetcher, drug_a)
#         future_b = executor.submit(fetch_all_data, fetcher, drug_b)

#         result_a = future_a.result()
#         result_b = future_b.result()

#     # Log results for both drugs
#     logging.info(f"Data for {drug_a}: {json.dumps(result_a, indent=2)}")
#     logging.info(f"Data for {drug_b}: {json.dumps(result_b, indent=2)}")

2025-01-22 03:53:50,375 - INFO - DrugBank XML successfully loaded.


### Structuring the data obtained and saving the knowledge graph as "drug_A_drug_b.ttl" file

In [19]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS
import json
import logging

# Define namespaces
BIO2RDF = Namespace("http://bio2rdf.org/")
DRUGBANK = Namespace("http://bio2rdf.org/drugbank:")
PUBCHEM = Namespace("http://bio2rdf.org/pubchem.compound:")
RXNORM = Namespace("http://bio2rdf.org/rxnorm:")
OPENFDA = Namespace("http://bio2rdf.org/openfda:")

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Initialize RDF graph
g = Graph()
def populate_knowledge_graph(result_a, result_b):

    g.bind("drugbank", DRUGBANK)
    g.bind("pubchem", PUBCHEM)
    g.bind("rxnorm", RXNORM)
    g.bind("openfda", OPENFDA)

    def add_drug_data(drug_data):
        if not drug_data:
            return
        drugbank_info = drug_data.get("drugbank", {})
        if drugbank_info:
            drug_id = drugbank_info.get("general_drug_info", {}).get("id")
            if drug_id:
                drug_uri = URIRef(DRUGBANK + drug_id)
                g.add((drug_uri, RDF.type, BIO2RDF.Drug))
                g.add((drug_uri, RDFS.label, Literal(drugbank_info["general_drug_info"]["name"])))

                # Add general drug details
                for key, value in drugbank_info["general_drug_info"].items():
                    if value:
                        g.add((drug_uri, DRUGBANK[key], Literal(value)))

                # Add product details
                for product in drugbank_info.get("matching_products", []):
                    product_uri = URIRef(DRUGBANK + product["product_name"].replace(" ", "_"))
                    g.add((drug_uri, DRUGBANK.hasProduct, product_uri))
                    g.add((product_uri, RDF.type, BIO2RDF.Product))
                    for key, value in product.items():
                        if value:
                            g.add((product_uri, DRUGBANK[key], Literal(value)))

        # Add PubChem data
        pubchem_data = drug_data.get("pubchem", {})
        if pubchem_data:
            for compound in pubchem_data.get("PC_Compounds", []):
                compound_id = compound.get("id", {}).get("id", {}).get("cid")
                if compound_id:
                    compound_uri = URIRef(PUBCHEM + str(compound_id))
                    g.add((drug_uri, PUBCHEM.hasCompound, compound_uri))

        # Add OpenFDA data
        openfda_data = drug_data.get("openfda", {})
        if openfda_data:
            for result in openfda_data.get("results", []):
                openfda_uri = URIRef(OPENFDA + result.get("id", "unknown"))
                g.add((drug_uri, OPENFDA.hasFDAData, openfda_uri))
                for key, value in result.items():
                    if isinstance(value, list):
                        for item in value:
                            g.add((openfda_uri, OPENFDA[key], Literal(item)))
                    elif value:
                        g.add((openfda_uri, OPENFDA[key], Literal(value)))

        # Add RxNorm data
        rxnorm_data = drug_data.get("rxnorm", {}).get("idGroup", {}).get("rxnormId")
        if rxnorm_data:
            for rxcui in rxnorm_data:
                rxcui_uri = URIRef(RXNORM + rxcui)
                g.add((drug_uri, RXNORM.hasRxCUI, rxcui_uri))

    # Add data for both drugs
    add_drug_data(result_a)
    add_drug_data(result_b)

def save_knowledge_graph(drug_a, drug_b):
    # Save the RDF graph to a file
    file_name = f"{drug_a}_{drug_b}.ttl"
    # Serialize graph to a file or directly to SPARQL endpoint
    g.serialize(destination=file_name, format="turtle")
    logging.info("Knowledge graph successfully populated and saved!")

def upload_rdf_to_fuseki(rdf_file):
    url = "http://localhost:3030/drugdb/data"
    headers = {"Content-Type": "text/turtle"}
    
    # Read the RDF content from the file
    with open(rdf_file, "rb") as file:
        rdf_data = file.read()

    # Send POST request with correct headers
    response = requests.post(url, data=rdf_data, headers=headers)
    
    if response.status_code == 200:
        print(f"Uploaded {rdf_file} successfully.")
        return True
    else:
        print(f"Failed to upload {rdf_file}. Status Code: {response.status_code}")
        print(f"Error: {response.text}")
        return False

#To test the above code, uncomment the below code
# if __name__ == "__main__":
#     # Initialize the DrugDataFetcher with the path to the DrugBank XML file
#     # drugbank_xml_path = "drugbank.xml"  # Replace with your actual path
#     # fetcher = DrugDataFetcher(drugbank_xml_path)

#     # Simulate user input for two drugs
#     drug_a = input("Enter the first drug or product name: ").strip()
#     drug_b = input("Enter the second drug or product name: ").strip()

#     # Use ThreadPoolExecutor to fetch data for both drugs in parallel
#     from concurrent.futures import ThreadPoolExecutor
#     with ThreadPoolExecutor() as executor:
#         future_a = executor.submit(fetch_all_data, fetcher, drug_a)
#         future_b = executor.submit(fetch_all_data, fetcher, drug_b)

#         result_a = future_a.result()
#         result_b = future_b.result()

#     # Log results for both drugs
#     logging.info(f"Data for {drug_a}: {json.dumps(result_a, indent=2)}")
#     logging.info(f"Data for {drug_b}: {json.dumps(result_b, indent=2)}")

#     # Populate the knowledge graph
#     populate_knowledge_graph(result_a, result_b)

#     #upload the RDF file to Fuseki server
#     rdf_file = f"{drug_a}_{drug_b}.ttl"
#     upload_rdf_to_fuseki(rdf_file)

### LLM Module : TO Fetch the RDF DATA and Summarize the Information

In [ ]:
from rdflib import Graph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


def read_ttl_file(file_path):
    """
    Read and parse the TTL file.
    """
    g = Graph()
    try:
        g.parse(file_path, format="turtle")
        logging.info(f"TTL file '{file_path}' successfully loaded.")
        return g
    except Exception as e:
        logging.error(f"Error loading TTL file '{file_path}': {e}")
        return None


def extract_data_from_graph(graph):
    """
    Extract relevant information from the RDF graph.
    """
    data = []
    for subject, predicate, obj in graph:
        data.append({
            "subject": str(subject),
            "predicate": str(predicate),
            "object": str(obj)
        })
    return data


def categorize_triples(data):
    """
    Categorize RDF triples into predefined categories for a structured summary based on observed schema.
    """
    categories = {
        "Drug Information": [],
        "Mechanism of Action": [],
        "Indications and Usage": [],
        "Dosage and Administration": [],
        "Side Effects and Toxicity": [],
        "Pregnancy and Breastfeeding": [],
        "Drug Interactions": [],
        "FDA Data": [],
        "Other Information": []
    }

    for triple in data:
        predicate = triple['predicate'].lower()

        if "mechanism_of_action" in predicate:
            categories["Mechanism of Action"].append(triple)
        elif "indication" in predicate or "usage" in predicate:
            categories["Indications and Usage"].append(triple)
        elif "dosage" in predicate or "administration" in predicate:
            categories["Dosage and Administration"].append(triple)
        elif "side_effect" in predicate or "toxicity" in predicate or "adverse_reaction" in predicate:
            categories["Side Effects and Toxicity"].append(triple)
        elif "pregnancy" in predicate or "breastfeeding" in predicate:
            categories["Pregnancy and Breastfeeding"].append(triple)
        elif "interaction" in predicate:
            categories["Drug Interactions"].append(triple)
        elif "fda" in predicate:
            categories["FDA Data"].append(triple)
        elif "description" in predicate or "label" in predicate or "name" in predicate:
            categories["Drug Information"].append(triple)
        else:
            categories["Other Information"].append(triple)

    return categories


def format_categorized_data(categories):
    """
    Format categorized data for summarization.
    """
    formatted_data = ""
    for category, triples in categories.items():
        formatted_data += f"{category}:\n"
        for triple in triples:
            formatted_data += f"  - {triple['subject']} - {triple['predicate']} - {triple['object']}\n"
        formatted_data += "\n"
    return formatted_data.strip()


def segment_text(input_text, max_tokens=1024):
    """
    Split input text into segments within the token limit.
    """
    words = input_text.split()
    segments = []
    for i in range(0, len(words), max_tokens):
        segment = " ".join(words[i:i + max_tokens])
        if segment.strip():
            segments.append(segment)
    return segments


def summarize_segments(segments, model, tokenizer, max_length=150, min_length=30):
    """
    Summarize each text segment and combine the results.
    """
    summaries = []
    for idx, segment in enumerate(segments):
        logging.info(f"Summarizing segment {idx + 1}/{len(segments)}")
        try:
            inputs = tokenizer(segment, return_tensors="pt", truncation=True, max_length=1024)
            summary_ids = model.generate(
                inputs["input_ids"],
                max_length=max_length,
                min_length=min_length,
                length_penalty=2.0,
                num_beams=4,
                early_stopping=True
            )
            summaries.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
        except Exception as e:
            logging.error(f"Error summarizing segment {idx + 1}: {e}")
            summaries.append(f"Error in segment {idx + 1}")
    return summaries


def combine_summaries(summaries):
    """
    Combine summaries from all segments into a final recommendation.
    """
    return "\n".join(summaries)


def generate_summary_with_model(prompt, model_name="facebook/bart-large-cnn"):
    """
    Use Hugging Face model directly to generate a summary.
    """
    try:
        # Load the model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

        # Segment the input text
        segments = segment_text(prompt, max_tokens=1024)

        # Summarize each segment
        summaries = summarize_segments(segments, model, tokenizer)

        # Combine the segment summaries
        final_summary = combine_summaries(summaries)

        return final_summary
    except Exception as e:
        logging.error(f"Error generating summary: {e}")
        return "Unable to generate a summary. Please check the input and try again."


#To test the functions, we can use the following code snippet:
# def main():
#     # Simulate user-selected TTL file
#     drug_a = input("Enter the first drug name: ").strip()
#     drug_b = input("Enter the second drug name: ").strip()
#     file_path = f"{drug_a}_{drug_b}.ttl"

#     # Step 1: Read the TTL file
#     graph = read_ttl_file(file_path)
#     if not graph:
#         logging.error("Could not load the TTL file. Exiting.")
#         return

#     # Step 2: Extract data from the RDF graph
#     extracted_data = extract_data_from_graph(graph)

#     # Step 3: Categorize and format data
#     categorized_data = categorize_triples(extracted_data)
#     formatted_data = format_categorized_data(categorized_data)

#     # Step 4: Summarize data using Hugging Face model
#     summary = generate_summary_with_model(formatted_data)

#     # Step 5: Display the summary
#     print("\n### Recommendation Summary ###")
#     print(summary)


# if __name__ == "__main__":
#     main()

c:\Users\Pranj\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
from drug_named_entity_recognition import find_drugs
from nltk.tokenize import word_tokenize

# Global variables to store the drug names
drug_a = ''
drug_b = ''

def extract_drug_names(user_input):
    global drug_a, drug_b  # Declare that we're modifying the global variables
    tokens = word_tokenize(user_input)
    drugs = find_drugs(tokens)
    drug_names = [drug[0]['name'] for drug in drugs]
    
    # Assign values to global variables
    if len(drug_names) > 0:
        drug_a = drug_names[0]
    if len(drug_names) > 1:
        drug_b = drug_names[1]

### For Testing (to validate the functionality and modify it)

In [ ]:
# def main():
#     # # Initialize the DrugDataFetcher with the path to the DrugBank XML file
#     # drugbank_xml_path = "drugbank.xml"  # Replace with your actual path
#     # fetcher = DrugDataFetcher(drugbank_xml_path)

#     # Simulate user input for two drugs
#     user_input = input("Enter Your Query: ")
#     extract_drug_names(user_input)

#     print(drug_a, drug_b)

#     # Use ThreadPoolExecutor to fetch data for both drugs in parallel
#     from concurrent.futures import ThreadPoolExecutor
#     with ThreadPoolExecutor() as executor:
#         future_a = executor.submit(fetch_all_data, fetcher, drug_a)
#         future_b = executor.submit(fetch_all_data, fetcher, drug_b)

#         result_a = future_a.result()
#         result_b = future_b.result()

#     # Log results for both drugs
#     logging.info(f"Data for {drug_a}: {json.dumps(result_a, indent=2)}")
#     logging.info(f"Data for {drug_b}: {json.dumps(result_b, indent=2)}")

#     # Populate the knowledge graph
#     populate_knowledge_graph(result_a, result_b)

#     # Save the RDF graph to a file
#     save_knowledge_graph(drug_a, drug_b)
    
#     #upload the RDF file to Fuseki server
#     rdf_file = f"{drug_a}_{drug_b}.ttl"
#     upload_rdf_to_fuseki(rdf_file)

#     #Read the TTL file
#     graph = read_ttl_file(rdf_file)
#     if not graph:
#         logging.error("Could not load the TTL file. Exiting.")
#         return
    
#     # Step 2: Extract data from the RDF graph
#     extracted_data = extract_data_from_graph(graph)

#     # Step 3: Format data for summarization
#     categorized_data = categorize_triples(extracted_data)
#     formatted_data = format_categorized_data(categorized_data)

#     # Step 4: Summarize data using Hugging Face model
#     summary = generate_summary_with_model(formatted_data)

#     # Step 5: Display the summary
#     print("\nRecommendation Summary:")
#     print(summary)
    
# if __name__ == "__main__":
#     main()

### GUI Module

In [27]:
import tkinter as tk
from tkinter import ttk, messagebox
import threading
import logging
import json
from concurrent.futures import ThreadPoolExecutor

# global variables
drug_a = ''
drug_b = ''

class DrugRecommendationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Drug Recommendation Tool")
        self.root.geometry("600x400")
        
        # Input Label
        self.query_label = ttk.Label(root, text="Enter your query:")
        self.query_label.pack(pady=10)
        
        # Input Entry Field
        self.query_entry = ttk.Entry(root, width=50)
        self.query_entry.pack(pady=5)
        
        # Submit Button
        self.submit_button = ttk.Button(root, text="Submit", command=self.start_process)
        self.submit_button.pack(pady=10)
        
        # Status Label
        self.status_label = ttk.Label(root, text="", font=("Arial", 12))
        self.status_label.pack(pady=10)
        
        # Result Section
        self.result_label = ttk.Label(root, text="Recommendation Summary:", font=("Arial", 14, "bold"))
        self.result_label.pack(pady=10)
        
        self.result_text = tk.Text(root, height=10, width=70, wrap=tk.WORD)
        self.result_text.pack(pady=5)

    def start_process(self):
        # Get the user query
        user_query = self.query_entry.get().strip()
        if not user_query:
            messagebox.showerror("Error", "Query cannot be empty!")
            return
        
        # Clear previous result and update status
        self.result_text.delete(1.0, tk.END)
        self.status_label.config(text="Fetching the information...")
        
        # Start the background process
        threading.Thread(target=self.run_main_process, args=(user_query,)).start()

    def run_main_process(self, user_query):
        global drug_a, drug_b
        
        try:
            # Step 1: Extract drug names
            extract_drug_names(user_query)
            
            # Step 2: Fetch data for both drugs
            with ThreadPoolExecutor() as executor:
                future_a = executor.submit(fetch_all_data, fetcher, drug_a)
                future_b = executor.submit(fetch_all_data, fetcher, drug_b)
                
                result_a = future_a.result()
                result_b = future_b.result()

            # Log results
            logging.info(f"Data for {drug_a}: {json.dumps(result_a, indent=2)}")
            logging.info(f"Data for {drug_b}: {json.dumps(result_b, indent=2)}")
            
            # Step 3: Populate the knowledge graph
            populate_knowledge_graph(result_a, result_b)
            
            # Step 4: Save the RDF graph and upload to Fuseki
            save_knowledge_graph(drug_a, drug_b)
            rdf_file = f"{drug_a}_{drug_b}.ttl"
            upload_rdf_to_fuseki(rdf_file)
            
            # Update status
            self.status_label.config(text="Information fetched, summarizing it now...")
            
            # Step 5: Read the TTL file
            graph = read_ttl_file(rdf_file)
            if not graph:
                messagebox.showerror("Error", "Failed to load the RDF file.")
                return
            
            # Step 6: Extract, categorize, and summarize data
            extracted_data = extract_data_from_graph(graph)
            categorized_data = categorize_triples(extracted_data)
            formatted_data = format_categorized_data(categorized_data)
            summary = generate_summary_with_model(formatted_data)
            
            # Display the result
            self.result_text.insert(tk.END, summary)
            self.status_label.config(text="Summary generated successfully.")
        
        except Exception as e:
            logging.error(f"An error occurred: {e}")
            messagebox.showerror("Error", f"An error occurred: {e}")

# Main Function
if __name__ == "__main__":
    root = tk.Tk()
    app = DrugRecommendationApp(root)
    root.mainloop()


2025-01-22 04:09:41,832 - INFO - Data for Acetaminophen: {
  "drugbank": {
    "general_drug_info": {
      "id": "DB00316",
      "name": "Acetaminophen",
      "description": "Acetaminophen (paracetamol), also commonly known as _Tylenol_, is the most commonly taken analgesic worldwide and is recommended as first-line therapy in pain conditions by the World Health Organization (WHO).[A176318] It is also used for its antipyretic effects, helping to reduce fever.[F4124] This drug was initially approved by the U.S. FDA in 1951 and is available in a variety of forms including syrup form, regular tablets, effervescent tablets, injection, suppository, and other forms.[L5756,L5774,F4124,Label]\r\n\r\nAcetaminophen is often found combined with other drugs in more than 600 over the counter (OTC) allergy medications, cold medications, sleep medications, pain relievers, and other products.[L5783] Confusion about dosing of this drug may be caused by the availability of different formulas, strengt

Uploaded Acetaminophen_Aspirin.ttl successfully.


2025-01-22 04:09:46,724 - INFO - Summarizing segment 1/22
2025-01-22 04:09:57,136 - INFO - Summarizing segment 2/22
2025-01-22 04:10:06,977 - INFO - Summarizing segment 3/22
2025-01-22 04:10:16,634 - INFO - Summarizing segment 4/22
2025-01-22 04:10:27,059 - INFO - Summarizing segment 5/22
2025-01-22 04:10:37,038 - INFO - Summarizing segment 6/22
2025-01-22 04:10:47,744 - INFO - Summarizing segment 7/22
2025-01-22 04:11:00,363 - INFO - Summarizing segment 8/22
2025-01-22 04:11:11,436 - INFO - Summarizing segment 9/22
2025-01-22 04:11:21,479 - INFO - Summarizing segment 10/22
2025-01-22 04:11:34,673 - INFO - Summarizing segment 11/22
2025-01-22 04:11:43,808 - INFO - Summarizing segment 12/22
2025-01-22 04:11:51,756 - INFO - Summarizing segment 13/22
2025-01-22 04:12:02,146 - INFO - Summarizing segment 14/22
2025-01-22 04:12:12,630 - INFO - Summarizing segment 15/22
2025-01-22 04:12:24,281 - INFO - Summarizing segment 16/22
2025-01-22 04:12:34,451 - INFO - Summarizing segment 17/22
2025-0